## Pivot Tables
The steps here parallel and produce same results as our ArcGISPro examples and running Pivot tables in Excel.

To see the code that generates this reproducible research example see the [GitHub repository](https://github.com/mhweber/UW_GIS_Material/blob/main/PivotTables.qmd). This notebook can be run as a Notebook within ArcPro and used for ‘literate programming’ where you fold you documentation, images, links, etc together with your code steps which can be passed along and re-run by others to produce the exact same results. 

## Enrich source GIS layers
### Generate jurisdictional information

- Read in necessary spatial layers from file geodatabase
- Combine the city and UGA layer with the county layer so that we have one dataset that represents all three jurisdictional types


## Using spatially enabled dataframes

- Read more about spatially enabled dataframes [here](https://developers.arcgis.com/python/guide/introduction-to-the-spatially-enabled-dataframe/)
- They add geometry to [pandas](https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://pandas.pydata.org/) dataframes in Python - they are an ESRI branding of [geopandas](https://geopandas.org)

In [41]:
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.geometry import SpatialReference

We can learn about methods on spatial data frames like this:

In [ ]:
help(pd.DataFrame.spatial.from_featureclass)

### Read each layer into a spatially enabled dataframe in Python

In [64]:
WRIA = pd.DataFrame.spatial.from_featureclass('WRIA', sr=SpatialReference(4326).as_arcpy)
WRIA.head()

,OBJECTID,WRIA_ID,WRIA_NR,WRIA_AREA_ACR_QT,WRIA_NM,SHAPE
0,1,3,1,1036786.459504,Nooksack,"{""rings"": [[[-122.41749579643245, 48.645405677..."
1,2,6,4,1567120.38494,Upper Skagit,"{""rings"": [[[-120.85224766453322, 49.000295000..."
2,3,7,48,1359169.774839,Methow,"{""rings"": [[[-120.75138542837956, 48.657128617..."
3,4,8,2,398410.613774,San Juan,"{""rings"": [[[-122.74885829556437, 48.722558501..."
4,5,11,3,472950.902479,Lower Skagit - Samish,"{""rings"": [[[-122.5890558775597, 48.6258125803..."


In [43]:
UGA = pd.DataFrame.spatial.from_featureclass('Whatcom_City_UGA', sr=SpatialReference(4326).as_arcpy)
UGA.head()

,OBJECTID,CITY_NM,UGA_NM,UGA_NM2,COUNTY_NM,GMA,FIPS_PLC,INCORP,ORIGIN,DATEMOD,SHAPE
0,1,Blaine,Blaine UGA,Blaine - Incorporated UGA,Whatcom,yes,06505,yes,Whatcom Co.,20160531,"{""rings"": [[[-122.77141447139361, 48.991232831..."
1,2,,Blaine UGA,Blaine - Unincorporated UGA,Whatcom,yes,,no,Whatcom Co.,20160531,"{""rings"": [[[-122.72962420321178, 48.971056279..."
2,3,Sumas,Sumas UGA,Sumas - Incorporated UGA,Whatcom,yes,68330,yes,Whatcom Co.,20160531,"{""rings"": [[[-122.26399132720081, 48.985610476..."
3,4,,Sumas UGA,Sumas - Unincorporated UGA,Whatcom,yes,,no,Whatcom Co.,20160531,"{""rings"": [[[-122.26399132720081, 48.985610476..."
4,5,,Birch Bay UGA,Birch Bay - Unincorporated UGA,Whatcom,yes,,no,Whatcom Co.,20160531,"{""rings"": [[[-122.77088227221428, 48.950687403..."


In [44]:
Fish = pd.DataFrame.spatial.from_featureclass('fish_distribution_whatcom', sr=SpatialReference(4326).as_arcpy)
Fish.head()

,OBJECTID_1,OBJECTID,LLID,LLID_STRM_,SPECIESRUN,SPECIES,RUNTIME_DE,DISTTYPE_D,USETYPE_DE,LIFEHIST_D,ShapeSTLen,Calc_Miles,SHAPE
0,1,4316,1207147487511,Park Creek,Dolly Varden/ Bull Trout,Bull Trout,Unknown or not Applicable,Documented,Rearing,Unknown,1355.421619,0.555574,"{""paths"": [[[-121.65228990461237, 48.721069111..."
1,2,4334,1207147487511,Park Creek,Coho,Coho Salmon,Unknown or not Applicable,No Gradient Barrier,Presence,Anadromous,6110.809033,2.506319,"{""paths"": [[[-121.68200782717946, 48.740809580..."
2,3,4370,1210637489118,Little Beaver Creek,Dolly Varden/ Bull Trout,Bull Trout,Unknown or not Applicable,Documented,Rearing,Unknown,7733.795459,3.162728,"{""paths"": [[[-121.06543769241031, 48.911189771..."
3,4,4385,1210597487735,Pierce Creek,Dolly Varden/ Bull Trout,Bull Trout,Unknown or not Applicable,Documented,Presence,Unknown,1675.215957,0.686268,"{""paths"": [[[-121.06593879264872, 48.772404142..."
4,5,4436,1207147487511,Park Creek,Dolly Varden/ Bull Trout,Bull Trout,Unknown or not Applicable,Documented,Rearing,Unknown,3211.176461,1.316875,"{""paths"": [[[-121.65885176685435, 48.727434245..."


## Union County and UGA

- first erase part of county with UGA polygons
- keep only fields interested in so field the same in both features
- then combine


In [45]:
arcpy.analysis.Erase(
    in_features="Whatcom_County",
    erase_features="Whatcom_City_UGA",
    out_feature_class=r"C:\Users\mwebe\UW Program\GIS550\Week5\Whatcom_Data_Tables.gdb\Whatcom_County_Erase",
    cluster_tolerance=None
)

<Result 'C:\\Users\\mwebe\\UW Program\\GIS550\\Week5\\Whatcom_Data_Tables.gdb\\Whatcom_County_Erase'>

In [46]:
arcpy.analysis.Union(
    in_features="Whatcom_County_Erase #;Whatcom_City_UGA #",
    out_feature_class=r"C:\Users\mwebe\UW Program\GIS550\Week5\Whatcom_Data_Tables.gdb\Whatcom_County_Union",
    join_attributes="ALL",
    cluster_tolerance=None,
    gaps="GAPS"
)

<Result 'C:\\Users\\mwebe\\UW Program\\GIS550\\Week5\\Whatcom_Data_Tables.gdb\\Whatcom_County_Union'>

In [75]:
Union = pd.DataFrame.spatial.from_featureclass("C:/Users/mwebe/UW Program/GIS550/Week5/Whatcom_Data_Tables.gdb/Whatcom_County_Union", sr=SpatialReference(4326).as_arcpy)
list(Union)

['OBJECTID_1', 'FID_Whatcom_County_Erase', 'OBJECTID', 'JURISDICT_', 'JURISDIC_1', 'JURISDIC_2', 'JURISDIC_3', 'JURISDIC_4', 'JURISDIC_5', 'JURISDIC_6', 'EDIT_DATE', 'EDIT_STATU', 'EDIT_WHO', 'FID_Whatcom_City_UGA', 'CITY_NM', 'UGA_NM', 'UGA_NM2', 'COUNTY_NM', 'GMA', 'FIPS_PLC', 'INCORP', 'ORIGIN', 'DATEMOD', 'SHAPE']

### Clean up fields

In [76]:
Union = Union.rename(columns={"JURISDIC_3": "Jurisdiction"})
Union = Union[["Jurisdiction","CITY_NM","UGA_NM","COUNTY_NM","SHAPE"]]
Union.head()

,Jurisdiction,CITY_NM,UGA_NM,COUNTY_NM,SHAPE
0,Whatcom County,,,,"{""rings"": [[[-122.757276202806, 49.00208319192..."
1,,Blaine,Blaine UGA,Whatcom,"{""rings"": [[[-122.7384683466864, 49.0020875296..."
2,,,Blaine UGA,Whatcom,"{""rings"": [[[-122.73620112522909, 49.002088444..."
3,,Sumas,Sumas UGA,Whatcom,"{""rings"": [[[-122.25130964930791, 49.002402619..."
4,,,Sumas UGA,Whatcom,"{""rings"": [[[-122.26399132720081, 48.985610476..."


### Assign Jurisdiction

In [77]:
Union['Jurisdiction'].loc[(Union['Jurisdiction'] == '') & (Union['CITY_NM'] != '')] = Union['CITY_NM']
Union.head()

,Jurisdiction,CITY_NM,UGA_NM,COUNTY_NM,SHAPE
0,Whatcom County,,,,"{""rings"": [[[-122.757276202806, 49.00208319192..."
1,Blaine,Blaine,Blaine UGA,Whatcom,"{""rings"": [[[-122.7384683466864, 49.0020875296..."
2,,,Blaine UGA,Whatcom,"{""rings"": [[[-122.73620112522909, 49.002088444..."
3,Sumas,Sumas,Sumas UGA,Whatcom,"{""rings"": [[[-122.25130964930791, 49.002402619..."
4,,,Sumas UGA,Whatcom,"{""rings"": [[[-122.26399132720081, 48.985610476..."


In [78]:
Union['Jurisdiction'].loc[(Union['Jurisdiction'] == '') & (Union['CITY_NM'] == '')] = Union['UGA_NM']
Union.head()

,Jurisdiction,CITY_NM,UGA_NM,COUNTY_NM,SHAPE
0,Whatcom County,,,,"{""rings"": [[[-122.757276202806, 49.00208319192..."
1,Blaine,Blaine,Blaine UGA,Whatcom,"{""rings"": [[[-122.7384683466864, 49.0020875296..."
2,Blaine UGA,,Blaine UGA,Whatcom,"{""rings"": [[[-122.73620112522909, 49.002088444..."
3,Sumas,Sumas,Sumas UGA,Whatcom,"{""rings"": [[[-122.25130964930791, 49.002402619..."
4,Sumas UGA,,Sumas UGA,Whatcom,"{""rings"": [[[-122.26399132720081, 48.985610476..."


### Now show unique values for Jurisdiction

In [79]:
Union['Jurisdiction'].unique()

<StringArray>
[     'Whatcom County',              'Blaine',          'Blaine UGA',
               'Sumas',           'Sumas UGA',       'Birch Bay UGA',
 'Columbia Valley UGA',            'Nooksack',        'Nooksack UGA',
    'Cherry Point UGA',          'Bellingham',      'Bellingham UGA',
            'Ferndale',          'Lynden UGA',              'Lynden',
             'Everson',        'Ferndale UGA',         'Everson UGA']
Length: 18, dtype: string

### Spatiallly join WRIA Information

In [80]:
WRIA = WRIA[['WRIA_NM','SHAPE']]
WRIA.head()

,WRIA_NM,SHAPE
0,Nooksack,"{""rings"": [[[-122.41749579643245, 48.645405677..."
1,Upper Skagit,"{""rings"": [[[-120.85224766453322, 49.000295000..."
2,Methow,"{""rings"": [[[-120.75138542837956, 48.657128617..."
3,San Juan,"{""rings"": [[[-122.74885829556437, 48.722558501..."
4,Lower Skagit - Samish,"{""rings"": [[[-122.5890558775597, 48.6258125803..."


In [94]:
County_UGA_WRIA = Union.spatial.join(WRIA)
County_UGA_WRIA = County_UGA_WRIA[["Jurisdiction","CITY_NM","UGA_NM","COUNTY_NM","WRIA_NM","SHAPE"]]
County_UGA_WRIA.head()

,Jurisdiction,CITY_NM,UGA_NM,COUNTY_NM,WRIA_NM,SHAPE
0,Whatcom County,,,,Nooksack,"{""rings"": [[[-122.757276202806, 49.00208319192..."
1,Blaine,Blaine,Blaine UGA,Whatcom,Nooksack,"{""rings"": [[[-122.7384683466864, 49.0020875296..."
2,Blaine UGA,,Blaine UGA,Whatcom,Nooksack,"{""rings"": [[[-122.73620112522909, 49.002088444..."
3,Sumas,Sumas,Sumas UGA,Whatcom,Nooksack,"{""rings"": [[[-122.25130964930791, 49.002402619..."
4,Sumas UGA,,Sumas UGA,Whatcom,Nooksack,"{""rings"": [[[-122.26399132720081, 48.985610476..."


### Clean up Fish fields

In [82]:
Fish = Fish[["SPECIES", "LLID_STRM_","DISTTYPE_D","USETYPE_DE","SHAPE"]]
Fish.head()

,SPECIES,LLID_STRM_,DISTTYPE_D,USETYPE_DE,SHAPE
0,Bull Trout,Park Creek,Documented,Rearing,"{""paths"": [[[-121.65228990461237, 48.721069111..."
1,Coho Salmon,Park Creek,No Gradient Barrier,Presence,"{""paths"": [[[-121.68200782717946, 48.740809580..."
2,Bull Trout,Little Beaver Creek,Documented,Rearing,"{""paths"": [[[-121.06543769241031, 48.911189771..."
3,Bull Trout,Pierce Creek,Documented,Presence,"{""paths"": [[[-121.06593879264872, 48.772404142..."
4,Bull Trout,Park Creek,Documented,Rearing,"{""paths"": [[[-121.65885176685435, 48.727434245..."


In [95]:
county_uga_wria_fish = Fish.spatial.join(County_UGA_WRIA)
county_uga_wria_fish.head()

,SPECIES,LLID_STRM_,DISTTYPE_D,USETYPE_DE,SHAPE,index_right,Jurisdiction,CITY_NM,UGA_NM,COUNTY_NM,WRIA_NM
0,Bull Trout,Park Creek,Documented,Rearing,"{""paths"": [[[-121.65228990461237, 48.721069111...",0,Whatcom County,,,,Nooksack
1,Coho Salmon,Park Creek,No Gradient Barrier,Presence,"{""paths"": [[[-121.68200782717946, 48.740809580...",0,Whatcom County,,,,Nooksack
2,Bull Trout,Little Beaver Creek,Documented,Rearing,"{""paths"": [[[-121.06543769241031, 48.911189771...",0,Whatcom County,,,,Nooksack
3,Bull Trout,Pierce Creek,Documented,Presence,"{""paths"": [[[-121.06593879264872, 48.772404142...",0,Whatcom County,,,,Nooksack
4,Bull Trout,Park Creek,Documented,Rearing,"{""paths"": [[[-121.65885176685435, 48.727434245...",0,Whatcom County,,,,Nooksack


In [99]:
county_uga_wria_fish['Calc_Miles'] = county_uga_wria_fish.SHAPE.geom.length.multiply(0.000189394)
county_uga_wria_fish.head()

,SPECIES,LLID_STRM_,DISTTYPE_D,USETYPE_DE,SHAPE,index_right,Jurisdiction,CITY_NM,UGA_NM,COUNTY_NM,WRIA_NM,Calc_Miles
0,Bull Trout,Park Creek,Documented,Rearing,"{""paths"": [[[-121.65228990461237, 48.721069111...",0,Whatcom County,,,,Nooksack,0.000002
1,Coho Salmon,Park Creek,No Gradient Barrier,Presence,"{""paths"": [[[-121.68200782717946, 48.740809580...",0,Whatcom County,,,,Nooksack,0.000009
2,Bull Trout,Little Beaver Creek,Documented,Rearing,"{""paths"": [[[-121.06543769241031, 48.911189771...",0,Whatcom County,,,,Nooksack,0.000013
3,Bull Trout,Pierce Creek,Documented,Presence,"{""paths"": [[[-121.06593879264872, 48.772404142...",0,Whatcom County,,,,Nooksack,0.000002
4,Bull Trout,Park Creek,Documented,Rearing,"{""paths"": [[[-121.65885176685435, 48.727434245...",0,Whatcom County,,,,Nooksack,0.000005


In [101]:
pivot = county_uga_wria_fish.pivot_table(index=['Jurisdiction','LLID_STRM_','SPECIES'],
                       values=['Calc_Miles'],
                       aggfunc='sum')
print(pivot)


                                                Calc_Miles
Jurisdiction   LLID_STRM_       SPECIES                   
Bellingham                      Bull Trout        0.000102
                                Chinook Salmon     0.00001
                                Chum Salmon       0.000095
                                Coho Salmon       0.000103
                                Cutthroat Trout   0.000106
...                                                    ...
Whatcom County Wiser Lake Creek Bull Trout        0.000102
                                Chum Salmon       0.000095
                                Coho Salmon       0.000123
                                Cutthroat Trout   0.000128
                                Steelhead Trout   0.000109

[1238 rows x 1 columns]
